In [1]:
import glob,os,numpy as np
import cPickle as pickle
#Please install Biopython via: pip install Biopython
import LibraryDesigner as ld
import LibraryTools as lt
from tqdm import tqdm_notebook as tqdm

In [155]:
reload(ld)

<module 'LibraryDesigner' from 'LibraryDesigner.py'>

### Construct count table with all the 17-mers in the genome/transcriptome

Warning: This requires >2*4**17  bytes (~34Gb) of free RAM

In [ ]:
reload(ld)
ct = ld.countTable(word=17,save_file=r'M:\genome_17w\hg38\allgenome_17w.npy',
                   sparse=False,fromfile=False,verbose=True)
ct.read(r'M:\genome_17w\hg38\human_genome.fasta')# can be file or list of files
ct.consume_batch(batch=1000000,reset=False)
ct.save()

### RNA only

#### Isoforms

In [ ]:
tr_file = r'M:\genome_17w\hg38\Homo_sapiens.GRCh38.cdna.all.fa'
names,seqs = ld.fastaread(tr_file)

In [ ]:
isoform_file = r'E:\Bogdan\Dropbox\code_Seurat\WholeGenome_MERFISH\MiscProbes\CBX5_isoforms.fasta'
names_is,seqs_is = zip(*[(nm,sq) for nm,sq in zip(names,seqs) if 'CBX5' in nm])
ld.fastawrite(isoform_file,names_is,seqs_is)

#### Design probes

In [36]:
reload(ld)

in_file = r'E:\Bogdan\Dropbox\code_Seurat\WholeGenome_MERFISH\MiscProbes\CBX5.fasta'
isoform_file = r'E:\Bogdan\Dropbox\code_Seurat\WholeGenome_MERFISH\MiscProbes\CBX5_isoforms.fasta'
save_file = in_file.replace('.fasta','.pbr')

transcriptome_fl = r'M:\genome_17w\hg38\transcriptome_17w.npy'
genome_fl = r'M:\genome_17w\hg38\allgenome_17w_new.npy'
rep_transcriptome_fl = r'M:\genome_17w\hg38\rtRNA_hg38.fasta'
#rep_genome_fl = r'/n/dulacfs2/Users/bbintu/Genomes/mouse/mm10/repeatSequences.fasta'
#local_genome_fl = in_files


pb_designer = ld.pb_reports_class(
    sequence_dic={'file':in_file,'use_revc':False,'use_kmer':True},
    map_dic={'transcriptome':{'file':transcriptome_fl,'use_revc':False,'use_kmer':True},
          #'genome':{'file':genome_fl,'use_revc':True,'use_kmer':True},
          'rep_transcriptome':{'file':rep_transcriptome_fl,'use_revc':True,'use_kmer':True},
          #'rep_genome':{'file':rep_genome_fl,'use_revc':True,'use_kmer':True},
          'isoforms':{'file':isoform_file,'use_revc':False,'use_kmer':True}},
          #'local_genome':{'file':local_genome_fl,'force_list':True,'use_revc':True,'use_kmer':True}},
    save_file=save_file,
    params_dic={'word_size':17,'pb_len':40,'buffer_len':2,'max_count':2**16-1,'check_on_go':False,'auto':False},
    dic_check={('transcriptome','isoforms'):10,
                'rep_transcriptome':0,'gc':[0.25,0.75],'tm':70})

pb_designer.computeOTmaps()
pb_designer.compute_pb_report()
pb_designer.perform_check_end()
pb_designer.plots()
pb_designer.save_csv(pb_designer,name='CBX5')

Setting attribute: map_isoforms
Mapping no. of seqs: 7
Setting attribute: map_rep_transcriptome
Mapping no. of seqs: 3866
Setting attribute: map_transcriptome
Time(s): 33.7380001545
Dealing with sequence: 1 out of 1
Time(s): 20.9230000973
Picking non-overlaping sequences.
Time(s): 0.744000196457


### Reverse complement, add stv1 readout sequence and select for IDT order

In [95]:
fl = r'E:\Bogdan\Dropbox\code_Seurat\WholeGenome_MERFISH\MiscProbes\CBX5.csv'
lines = np.array([ln[:-1].split(',') for ln in open(fl,'r')][1:])
seqs,names = lines[:,:2].T
npbs = 50
inds = np.linspace(0,len(seqs)-1,npbs+2)[1:-1].astype(int)
names = names[inds]
stv1_25mer = r'CGCAACGCTTGGGACGGTTCCAATC'
tag = lt.seqrc(stv1_25mer)[-20:].lower()
seqs = [tag+lt.seqrc(sq) for sq in seqs[inds]]

for nm,sq in zip(names,seqs):
    print ','.join([nm+'_stv1',sq,'25nm','STD'])

CBX5_pb_168_stv1,gaaccgtcccaagcgttgcgGCATGCAGTTAAGCATTGTGGCTATGGCCTCCCCTGCCTC,25nm,STD
CBX5_pb_405_stv1,gaaccgtcccaagcgttgcgCTTTCCACTTCAGTAGATATTCCACTTGTCCCTTAACCAC,25nm,STD
CBX5_pb_590_stv1,gaaccgtcccaagcgttgcgTTTAGATTTGATGTCATCGGCACTGTTTGAGAAATTGGAT,25nm,STD
CBX5_pb_800_stv1,gaaccgtcccaagcgttgcgCCATGTCAGTCTCTCTTCATAAAATGCTATCACAATTTGT,25nm,STD
CBX5_pb_1010_stv1,gaaccgtcccaagcgttgcgCAAGAGAACCAATACCAACATTTCTCCTGTGGAGCACAGT,25nm,STD
CBX5_pb_1178_stv1,gaaccgtcccaagcgttgcgCAAGACTCTAAGGTGATTAAAAAGTTGAAACTCATCTTGA,25nm,STD
CBX5_pb_1388_stv1,gaaccgtcccaagcgttgcgGGGTAGAGCTGAACAAGGTTGGGAAAGCCCCAACAGGTAG,25nm,STD
CBX5_pb_1635_stv1,gaaccgtcccaagcgttgcgACTATAGCTACAAATCTGCACAGTTAATAATCTTGTCATT,25nm,STD
CBX5_pb_1805_stv1,gaaccgtcccaagcgttgcgAAAAGTTGGCAGGGTTGGAGGGTGCCTAGCACCTTGGCAC,25nm,STD
CBX5_pb_2015_stv1,gaaccgtcccaagcgttgcgATATTTTGGGGACTCTTGGTGGTAAATCCAGGAGCCAAGA,25nm,STD
CBX5_pb_2330_stv1,gaaccgtcccaagcgttgcgAAGAAACCAAAACGAAGTATTTTGCTGAGTAAGAAGAAAA,25nm,STD
CBX5_pb_2543_stv1,gaaccgtcccaagcgttg